# Outline of Notebook

* Theory of Bayesian Knowledge Tracing (BKT)
* Generating Synthetic Student Data According to BKT Assumptions
* Generating Synthetic Student Data According to IRT Assumptions
* Python Code for Generating Synthetic Data According to BKT Assumptions
* Fitting an HMM to Synthetic Student Data
 * The Label Problem
* Python Code for Fitting an HMM to Synethetic Student Data
* Comparison with Matlab

# Theory of Bayesian Knowledge Tracing (BKT)

## Inference

The probability of a student getting a question correct at time opportunity $n$ is:

$$P(\text{correct}_n) = P(L_n) \cdot (1 - P(S)) + (1 - P(L_n)) \cdot P(G)$$

The probability of a student getting a question incorrect at time opportunity $n$ is:

$$P(\text{incorrect}_n) = P(L_n) \cdot P(S) + (1 - P(L_n)) \cdot (1 - P(G))$$

## Fitting the Model: Part 1

There are two distinct stages to using the knowledge tracing model. The first stage is to fit the model parameters from data. The second stage is using the model to infer the student’s knowledge over time given the student’s responses. Given an observation of the student’s response at time opportunity n (correct or incorrect) the probability
P(Ln) that a student knows the skill is calculated using Bayes rule. When a correct response is observed, this probability is as follows:

$$P(L_n | \text{correct}_n) = \frac{P(L_n) \cdot (1 - P(S))}{P(\text{correct}_n)}$$

When an incorrect response is observed, this probability is as follows:

$$P(L_n | \text{incorrect}_n) = \frac{P(L_n) \cdot P(S)}{P(\text{incorrect}_n)}$$

## Fitting the Model: Part 2

Lastly, we show how the student’s knowledge of the skill is updated given its interaction with the system. This estimate is the sum of two probabilities: the posterior probability that the student already knew the skill (contingent on the evidence), and the probability that the student did not know the skill, but was able to learn it.

$$P(L_n) = P(L_{n-1} | \text{evidence}_{n-1}) + (1 - P(L_{n-1} | \text{evidence}_{n-1})) \cdot P(T)$$




# Generating Synthetic Data According to BKT Assumptions

To generate student data according to BKT assumptions, we start by deciding on the number of concepts that our synthetic student will learn from. In our running example, the students will answer questions on a single topic. Next, we set the true values for the parameters $P(L_0)$, $P(T)$, $P(G)$, and $P(S)$. Here, we choose $P(L_0) = 0.1$, $P(T) = 0.2$, $P(G) = 0.25$, and $P(S) = 0.1$. (If we were dealing with multiple concepts, we would choose a different set of values for each concept.) Finally, once we have the parameters, we generate student responses on a sequence of questions; in this example, 20 questions on a single topic.

We generate student data in thre steps.

1. Determine the probability of getting a question correct with the formula $P(\text{correct}_n) = P(L_n) \cdot (1 - P(S)) + (1 - P(L_n)) \cdot P(G)$
1. Draw an answer from a Bernoulli distribution with mean = $P(\text{correct}_n)$.
1. Update the probability of knowing the concept depending on whether the answer was correct or incorrect

In [2]:
from __future__ import division
import numpy as np
import csv
from hmmlearn import hmm

def generate_bkt_responses(numStudents, numQuestions, pL0=0.05, pT=0.1, pGuess=0.25, pSlip=0.05):
    
    all_answers = []
    for i in range(numStudents):
        pL = np.zeros(numQuestions)
        pL[0] = pL0
        answers = np.zeros(numQuestions)
        for i in range(numQuestions):

            # Answer ith question according to probability of knowing the concept at step i
            pCorrect = (pL[i] * (1 - pSlip)) + ((1 - pL[i]) * pGuess)
            cur_answer = np.random.binomial(1, pCorrect)
            answers[i] = cur_answer

            if i == numQuestions-1: break

            # Update the probability of knowing the concept conditioned on whether the answer
            # was correct or incorrect. 
            if cur_answer == 1:
                numerator = pL[i] * (1 - pSlip)
                denominator = numerator + ((1 - pL[i]) * pGuess)
                pL[i+1] = (numerator / denominator)
            elif cur_answer == 0:
                numerator = pL[i] * pSlip
                denominator = numerator + ((1 - pL[i]) * (1 - pGuess))
                pL[i+1] = (numerator / denominator)

            # Finally, update the probability of knowing the concept given the response
            pL[i+1] = pL[i+1] + ((1 - pL[i+1]) * pT)
       
        answers = [int(a) for a in answers]
        all_answers.append(answers)
    return all_answers

def print_bkt_params(hmm_fit):
    print "pT is: %f" % hmm_fit.transmat_[0,1]
    print "pGuess is: %f" % hmm_fit.emissionprob_[0,1]
    print "pSlip is: %f" % hmm_fit.emissionprob_[1,0]
    print "pL0 is: %f" % hmm_fit.startprob_[1]


# Generating Synthetic Student Data According to IRT Assumptions


Simulated Data: We simulate virtual students learning virtual concepts and test how well we can
predict responses in this controlled setting. For each run of this experiment we generate two thousand students who answer 50 exercises drawn from $k$ concepts. For this dataset only, all students answer the same sequence of 50 exercises. Each student has a latent knowledge state "skill" for each concept, and each exercise has both a single concept and a difficulty. The probability of a student getting a exercise with difficulty β correct if the student had concept skill α is modelled using classic Item Response Theory as: 

$$P(\text{correct} | \alpha, \beta) = c + \frac{1-c}{1 + e^{\beta - \alpha}}$$

where $c$ is the probability of a random guess, set to be 0.25. 

# Fitting an HMM to Synthetic Student Data

Once we have generated synthetic student data, we can re-learn the parameters used to generate the data using an HMM. Here we train a multinomial HMM using the Python package HMMLearn.

We will learn the following sets of parameters:

* Emission probabilities
* Transmission probabilities
* Start state probability

Which will take the following form:

![title](images/bkt_params.png)

## The Label Switching Problem
There is something called the Label-Switching Problem that we need to be careful of. Essentially, the rows and columns of the output probability matrices can be in any order. For example, there is no guarantee that the 0th row of the emission probability matrix corresponds to the 0th state. This means that we need a heuristic for determining the row and column labels after we learn HMM parameters.

More info here:
* https://github.com/hmmlearn/hmmlearn/issues/106
* http://stackoverflow.com/questions/39756006/how-to-map-hidden-states-to-their-corresponding-categories-after-decoding-in-hmm
* https://www.google.com/#q=label-switching+problem

## The Label Switching Problem: Not a Problem in Matlab
It appears that the label-switching problem does not occur in Matlab. No matter how I generate synthetic data (either in Python or Matlab), Matlab consistently outputs the parameter matrices in the same order. This is a strong reason to consider switching to Matlab for BKT.

In [8]:
# Fit BKT parameters using synthetic student data

# Set true parameter values for generating synthetic data
pL0=0.1
pT=0.2
pG=0.25
pS=0.05

# Generate synthetic data and format correctly
numStudents = 100
numQuestions = 20
answers = generate_bkt_responses(numStudents, numQuestions, pL0=pL0, pT=pT, pGuess=pG, pSlip=pS)
lengths = [len(i) for i in answers]
X = np.concatenate(answers)
X = np.atleast_2d(X).T

# Learn parameters from synthetic data
bkt_model = hmm.MultinomialHMM(n_components=2)
bkt_fit = bkt_model.fit(X, lengths)

print "The true value of pT is %.02f. The transmission matrix probability is: " % pT
print bkt_fit.transmat_
print "--------------------"
print "The true value of pG is %.02f." % pG
print "The true value of pS is %.02f. THe emission probability matrix is: " % pS
print bkt_fit.emissionprob_
print "--------------------"
print "The true value of pL0 is %.02f. The start probability matrix is:  " % pL0
print bkt_fit.startprob_
print "--------------------"

The true value of pT is 0.20. The transmission matrix probability is: 
[[ 0.61733849  0.38266151]
 [ 0.16558054  0.83441946]]
--------------------
The true value of pG is 0.25.
The true value of pS is 0.05. THe emission probability matrix is: 
[[ 0.55118856  0.44881144]
 [ 0.02975853  0.97024147]]
--------------------
The true value of pL0 is 0.10. The start probability matrix is:  
[ 0.98676419  0.01323581]
--------------------


# Generate Synthetic Data in Matlab and Train Parameters in Python

It is simple to specify the parameters for an HMM in Matlab and generate sequences. In order to validate the code I wrote here to learn HMM params, I created an HMM with the same "true" paramters as here in Matlab and generated synthetic student data. Below I read in that data and learn the parameters using Python.

Currently, the parameters used to generate data in Python are the same as those used to generate data in Matlab.

In [6]:
# Read in data generated from an HMM in Matlab. 
# Currently, the parameters used in Matlab are the same as those used here.
matlab_file = '../data/matlab_synthetic.csv'
answers = []
lengths = []
with open(matlab_file, 'rb') as csvfile:
    data_reader = csv.reader(csvfile)
    for row in data_reader:
        new_row = [int(i)-1 for i in row]
        answers.append(new_row)
        lengths.append(len(row))
X = np.concatenate(answers)
X = np.atleast_2d(X).T

# Initial guesses (prior beliefs) of HMM parameters
trans = np.array([[0.6,0.4],
                  [0,1]])
start = np.array([0.9, 0.1])
# For some reason, the emission probabilities can't be specified in HMMLearn (but they can in Matlab)

bkt_model = hmm.MultinomialHMM(n_components=2, transmat_prior=trans, startprob_prior=start)
bkt_fit = bkt_model.fit(X, lengths, )

print "The transmission matrix probability is: "
print bkt_fit.transmat_
print "--------------------"
print "The emission probability matrix is: "
print bkt_fit.emissionprob_
print "--------------------"
print "The start probability matrix is: "
print bkt_fit.startprob_
print "--------------------"

The transmission matrix probability is: 
[[ 0.52290758  0.47709242]
 [ 0.50307994  0.49692006]]
--------------------
The emission probability matrix is: 
[[ 0.13947199  0.86052801]
 [ 0.22456961  0.77543039]]
--------------------
The start probability matrix is: 
[ 0.24379197  0.75620803]
--------------------


# Generate Synthetic Data in Python and Train Parameters in Matlab

In order to validate the Python method I used generate synthetic student data (which should be the same as the Matlab method), I write the synthetic student data to a csv file, load it in Matlab, and learn the HMM params using Matlab's HMM toolbox.

Currently, the parameters used to generate data in Python are the same as those used to generate data in Matlab (and vice versa).

In [4]:
# Write answers to CSV and learn parameters in Matlab
with open('../data/python_synthetic.csv', 'wb') as csvfile:
    writer = csv.writer(csvfile)
    for a in answers:
        newrow = [i+1 for i in a]
        writer.writerow(newrow)

## Matlab's Learned Params

Matlab always outputs the transmission and emission probability matrices in the same order. Here is one example:

    estTR =

    0.8022    0.1978
         0    1.0000


    estE =

    0.7733    0.2267
    0.0515    0.9485
    
The true values used to generate the data in Python were pL0=0.1, pT=0.2, pG=0.25, and pS=0.05. Here we see those values are estimated by Matlab to be pT-hat=0.20, pG-hat=0.23, and pS-hat=0.05. Note that Matlab does not return pL0.